In [6]:
import tensorflow as tf
import tensorflow_datasets as tfds

/Users/benhicks/Documents/UW/mlops-tooling/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

Dl Completed...: 100%|██████████| 5/5 [00:03<00:00,  1.57 file/s]

Dataset mnist downloaded and prepared to /Users/benhicks/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


In [8]:
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

In [9]:
ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)

In [11]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10)
])
model.compile(
    optimizer=tf.keras.optimizers.legacy.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

model.fit(
    ds_train,
    epochs=6,
    validation_data=ds_test,
)

Epoch 1/6
469/469 [==============================] - 5s 9ms/step - loss: 0.4103 - sparse_categorical_accuracy: 0.8834 - val_loss: 0.2918 - val_sparse_categorical_accuracy: 0.9166
Epoch 2/6
469/469 [==============================] - 4s 9ms/step - loss: 0.2929 - sparse_categorical_accuracy: 0.9178 - val_loss: 0.2811 - val_sparse_categorical_accuracy: 0.9236
Epoch 3/6
469/469 [==============================] - 4s 9ms/step - loss: 0.2825 - sparse_categorical_accuracy: 0.9213 - val_loss: 0.2838 - val_sparse_categorical_accuracy: 0.9200
Epoch 4/6
469/469 [==============================] - 4s 9ms/step - loss: 0.2788 - sparse_categorical_accuracy: 0.9234 - val_loss: 0.2823 - val_sparse_categorical_accuracy: 0.9210
Epoch 5/6
469/469 [==============================] - 4s 9ms/step - loss: 0.2794 - sparse_categorical_accuracy: 0.9215 - val_loss: 0.2864 - val_sparse_categorical_accuracy: 0.9237
Epoch 6/6
469/469 [==============================] - 4s 8ms/step - loss: 0.2785 - sparse_categorical_accu

In [14]:
from mlops_tooling import ModelManager

In [15]:
metrics = {
    "loss": 0.2785,
    "sparse_categorical_accuracy": 0.9219,
    "val_loss": 0.2963,
    "val_sparse_categorical_accuracy": 0.9190
}

In [18]:
LOCATION = "europe-west2"
API_ENDPOINT = "europe-west2-aiplatform.googleapis.com"
TRACKING_URI = "https://mlflow.dev.gcp.uw.systems/"
GOOGLE_PROJECT = 'uw-data-science-dev'
GCS_BUCKET =  "gs://uw-data-science-model-artifacts"

mlflow = ModelManager(
    tracking_uri = TRACKING_URI,
    google_project=GOOGLE_PROJECT,
    gcs_bucket=GCS_BUCKET,
    
    serving_container_image_uri="europe-docker.pkg.dev/vertex-ai/prediction/tf-cpu.2-11.py310:latest"
)

In [19]:
experiment_name = "MNIST Test"
mlflow.create_new_experiment(experiment_name)

In [20]:
mlflow.log_results(
    experiment_name=experiment_name,
    run_name="04-07-2023_MNIST_Test",
    model=model,
    parameters={},
    metrics=metrics,
    model_type='tensorflow'
)

2023/07/04 09:34:59 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: /var/folders/xy/2pcnq4kj58xfn6t2h3zfng080000gp/T/tmpfr0fauxz/model/data/model/assets


INFO:tensorflow:Assets written to: /var/folders/xy/2pcnq4kj58xfn6t2h3zfng080000gp/T/tmpfr0fauxz/model/data/model/assets
/Users/benhicks/Documents/UW/mlops-tooling/.venv/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
